
<h1 align=center> Image Super Resolution using Autoencoders</h1>

<img src="images/high_res_v_low_res.jpg" width=550px>

## Task 1: Project Overview and Import Libraries

In [1]:
import os
import re
from scipy import ndimage, misc
from skimage.transform import resize, rescale
from matplotlib import pyplot
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(0)

from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.layers import Conv2DTranspose, UpSampling2D, add
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
import tensorflow as tf
print(tf.__version__)

2.2.0


## Task 2: What are Autoencoders?

<img src="images/autoencoder.jpg">
Credit: Autoencoder Schema by <a href="https://blog.keras.io/img/ae/autoencoder_schema.jpg">Francois Chollet, 2016</a>.

<h4 align=center>Encoder Architecture</h4>
<img src="/home/ravineel/Desktop/Github/MDR_Projects/Image_Super_Scaling/Imageencoder.png" width=450px align=center>

## Task 3: Build the Encoder

In [2]:
input_img = Input(shape=(256,256,3))
l1 = Conv2D(64,(3,3),padding='same',activation='relu',activity_regularizer=regularizers.l1(10e-10))(input_img)
l2 = Conv2D(64,(3,3),padding='same',activation='relu',activity_regularizer=regularizers.l1(10e-10))(l1)
l3 = MaxPooling2D(padding='same')(l2)
l4 = Conv2D(128,(3,3),padding='same',activation='relu',activity_regularizer=regularizers.l1(10e-10))(l3)
l5 = Conv2D(128,(3,3),padding='same',activation='relu',activity_regularizer=regularizers.l1(10e-10))(l4) 
l6 = MaxPooling2D(padding='same')(l5)
l7 =Conv2D(256,(3,3),padding='same',activation='relu',activity_regularizer=regularizers.l1(10e-10))(l6) 
encoder = Model(input_img,l7,name="Encoder")

In [3]:
encoder.summary()

Model: "Encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 256, 256, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 256, 256, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 128, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 128, 128, 128)     147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 128)       0   

## Task 4: Build the Decoder to Complete the Network

<img src="images/decoder.png" width=450px>

In [4]:
input_img = Input(shape=(256,256,3))
l1 = Conv2D(64,(3,3),padding='same',activation='relu',activity_regularizer=regularizers.l1(10e-10))(input_img)
l2 = Conv2D(64,(3,3),padding='same',activation='relu',activity_regularizer=regularizers.l1(10e-10))(l1)
l3 = MaxPooling2D(padding='same')(l2)
l4 = Conv2D(128,(3,3),padding='same',activation='relu',activity_regularizer=regularizers.l1(10e-10))(l3)
l5 = Conv2D(128,(3,3),padding='same',activation='relu',activity_regularizer=regularizers.l1(10e-10))(l4) 
l6 = MaxPooling2D(padding='same')(l5)
l7 =Conv2D(256,(3,3),padding='same',activation='relu',activity_regularizer=regularizers.l1(10e-10))(l6) 

In [5]:
l8 =  UpSampling2D()(l7)
l9 =  Conv2D(128,(3,3),padding='same',activation='relu',activity_regularizer=regularizers.l1(10e-10))(l8)
l10 =  Conv2D(128,(3,3),padding='same',activation='relu',activity_regularizer=regularizers.l1(10e-10))(l9)
l11 = add([l5,l10])
l12 =  UpSampling2D()(l11)
l13 =  Conv2D(64,(3,3),padding='same',activation='relu',activity_regularizer=regularizers.l1(10e-10))(l12)
l14 =  Conv2D(64,(3,3),padding='same',activation='relu',activity_regularizer=regularizers.l1(10e-10))(l13)
l15 = add([l14,l2])
decoder = Conv2D(3,(3,3),padding='same',activity_regularizer=regularizers.l1(10e-10),activation='relu')(l15)
autoencoder =Model(input_img,decoder,name="Autoencoder")
autoencoder.summary()

Model: "Autoencoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 256, 256, 64) 1792        input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 256, 256, 64) 36928       conv2d_5[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_2 (MaxPooling2D)  (None, 128, 128, 64) 0           conv2d_6[0][0]                   
________________________________________________________________________________________

In [6]:
autoencoder.compile(optimizer='adadelta', loss='mean_squared_error')

## Task 5: Create Dataset and Specify Training Routine

In [7]:
def train_batches(just_load_dataset=False):

    batches = 256 

    batch = 0 
    batch_nb = 0 
    max_batches = -1 
    
    ep = 4 

    images = []
    x_train_n = []
    x_train_down = []
    
    x_train_n2 = [] 
    x_train_down2 = []
    
    for root, dirnames, filenames in os.walk("/home/ravineel/Desktop/Github/MDR_Projects/Image_Super_Scaling/Data"):
        for filename in filenames:
            if re.search("\.(jpg|jpeg|JPEG|png|bmp|tiff)$", filename):
                if batch_nb == max_batches: 
                    return x_train_n2, x_train_down2
                filepath = os.path.join(root, filename)
                image = pyplot.imread(filepath)
                if len(image.shape) > 2:
                        
                    image_resized = resize(image, (256, 256))
                    x_train_n.append(image_resized)
                    x_train_down.append(rescale(rescale(image_resized, 2.0), 0.5))
                    batch += 1
                    if batch == batches:
                        batch_nb += 1

                        x_train_n2 = np.array(x_train_n)
                        x_train_down2 = np.array(x_train_down)
                        
                        if just_load_dataset:
                            return x_train_n2, x_train_down2
                        
                        print('Training batch', batch_nb, '(', batches, ')')

                        autoencoder.fit(x_train_down2, x_train_n2,
                            epochs=ep,
                            batch_size=10,
                            shuffle=True,
                            validation_split=0.15,  )
                    
                        x_train_n = []
                        x_train_down = []
                    
                        batch = 0

    return x_train_n2, x_train_down2

## Task 6: Load the Dataset and Pre-trained Model

In [18]:
x_train_n, x_train_down = train_batches(just_load_dataset=True)

Training batch 1 ( 256 )
Epoch 1/4
22/22 [==============================] - 193s 9s/step - loss: 0.0070 - val_loss: 0.0060
Epoch 2/4
22/22 [==============================] - 189s 9s/step - loss: 0.0067 - val_loss: 0.0057
Epoch 3/4
22/22 [==============================] - 190s 9s/step - loss: 0.0064 - val_loss: 0.0055
Epoch 4/4
14/22 [==================>...........] - ETA: 1:05 - loss: 0.0061

In [ ]:
autoencoder.load_weights("/home/ravineel/Desktop/Github/MDR_Projects/Image_Super_Scaling/Data/sr.img_net.mse.final_model5.no_patch.weights.best.hdf5")

## Task 7: Model Predictions and Visualizing the Results

In [ ]:
encoder.load_weights('/home/ravineel/Desktop/Github/MDR_Projects/Image_Super_Scaling/Data/encoder_weights.hdf5')

In [ ]:
encoded_img= encoder.predict(x_train_down)

In [ ]:
encoded_img.shape

In [ ]:
sr1 = np.clip(autoencoder.predict(x_train_down),0.0,1.0)

In [ ]:
image_index= 20

In [ ]:
plt.figure(figsize=(128,128))
i=1
ax = plt.subplot(10,10 ,i)
plt.imshow(x_train_down[image_index])
i=i+1
ax = plt.subplot(10,10 ,i)
plt.imshow(x_train_down[image_index], interpolation='bicubic')
i=i+1
ax = plt.subplot(10,10 ,i)
plt.imshow(encoded_img[image_index].reshape((64*64,256)))
i=i+1
ax = plt.subplot(10,10 ,i)
plt.imshow(sr1[image_index])
i=i+1
ax = plt.subplot(10,10 ,i)
plt.imshow(x_train_n[image_index])
plt.show()